# Cosmos DB Demos

This notebook has demos for two containers in two different databases:

| Database | Container | Description |
| - | - | - |
| Families | Families | 3 simple family documents |
| adventure-works | stores | 701 documents imported from AdventureWorks |

## Documentation

https://docs.microsoft.com/en-us/python/api/azure-cosmos

https://docs.microsoft.com/en-us/azure/cosmos-db/use-notebook-features-and-commands

## Import External Modules

### json
* Python JSON Encoder/Decoder.
  * We'll be using the `json.dumps` function with `indent` to "pretty-print" JSON output.
### pandas
* Python Data Analysis Library ("panel data analysis")
  * This library provides several ways to visualize query results in a Python dataframe

In [1]:
import json
import pandas as pd

## Initialize Database and Container References
To do anything, we'll need a **database** reference, and a reference to **container** within that database.

This code uses the `cosmos_client` that's built in to all Cosmic Notebooks as part of the Python SDK for the Core API. We call `get_database_client` to obtain a reference to the **Families** and **mydb** database, and then call `get_container_client` on those databases to obtain a reference to the **Families** and **mystore** containers (respectively) in those databases.

In [2]:
# Families database, Families container
familiesDb = cosmos_client.get_database_client('Families')
families = familiesDb.get_container_client('Families')

# adventure-works database, stores container
adventureworksDb = cosmos_client.get_database_client('adventure-works')
stores = adventureworksDb.get_container_client('stores')

## Define Query Function
We'll call this function to execute a SQL query against a container and return the results in a `list`. We'll also permit cross-partition queries by including the `enable_cross_partition_query=True` option.

In [3]:
# Execute a query and display the results as JSON
def query(container, sql, partitionKey = None, desc = None, showStats = True):
    try:
        items = container.query_items(
            query = sql,
            enable_cross_partition_query = True,
            partition_key = partitionKey)
        result = list(items)
        first10 = result[0:10]
        if desc != None:
            print (desc)
        if showStats:
            count = len(result)
            charge = container.client_connection.last_response_headers['x-ms-request-charge']
            print (count, 'document(s) selected (charge=', charge, 'RUs)\n')
        print (json.dumps(first10, indent=4))
    except Exception as e:
        print(e)